## VAISHNAVI JAMDADE(TM39453)
### HOMEWORK 4

### Importing required libraries:

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import idx2numpy
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import accuracy_score

### Converting the file into numpy array of dimensions (60000,28,28)

In [2]:
def convertfunc(file):
    arr = idx2numpy.convert_from_file(file)
    return arr

In [3]:
X_train=convertfunc('train-images.idx3-ubyte')
y_train=convertfunc('train-labels.idx1-ubyte')
X_test=convertfunc('t10k-images.idx3-ubyte')
y_test=convertfunc('t10k-labels.idx1-ubyte')

In [4]:
X_train.shape

(60000, 28, 28)

In [5]:
X_test.shape, y_train.shape

((10000, 28, 28), (60000,))

### Reshaping the dataset :

In [6]:
def reshapefunc(array,rows,columns):
    array=array.reshape(rows,columns)
    return array

In [7]:
X_train=reshapefunc(X_train,60000,784)
X_test=reshapefunc(X_test,10000,784)
#y_train=reshapefunc(y_train,1,60000)
#y_test=reshapefunc(y_test,1,10000)

In [8]:
X_train.shape,X_test.shape, y_train.shape,y_test.shape

((60000, 784), (10000, 784), (60000,), (10000,))

### Using One hot label encoding for output labels:

In [9]:
y_train = pd.get_dummies(y_train)
y_train=np.array(y_train)

In [10]:
y_train

array([[0, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 1, 0]], dtype=uint8)

In [11]:
X_train.shape, X_test.shape

((60000, 784), (10000, 784))

### Initializing weights and bias:

In [12]:
def initialize_weights(inputs, hNeurons, oNeurons):

    weights1=np.random.randn(inputs,hNeurons)*np.sqrt(1./inputs)    #784 rows 5 columns
    bias1=np.zeros((1, hNeurons))*np.sqrt(1./inputs)
    #hence x.w = 1,5 dimensions + bias [1,5]
    weights2=np.random.randn(hNeurons,oNeurons)*np.sqrt(1./hNeurons)
    #hence x.w= 5,10 dimensions + bias [1,10]
    bias2= np.zeros((1, oNeurons))*np.sqrt(1./hNeurons)

    #ouptut vecotr is going to be of dimensions [1,10] then softmax
    #W1.shape , b1.shape, W2.shape, b2.shape
    
    return weights1, bias1, weights2, bias2


### Sigmoid Activation Function:

In [13]:
def sigmoid(z):
    act = 1. / (1. + np.exp(-z))
    return act

### Softmax Activation function :

In [14]:
#Normalising the values:

def softmax(z):
    softact = np.exp(z)/np.sum(np.exp(z), axis=1,keepdims=True)                                       
    return softact

'''
def softmax(z):
    softact = np.exp(z - np.max(z, axis=1, keepdims=True))
    softact=softact/np.sum(softact, axis=1, keepdims=True)
    
    return softact
'''

'\ndef softmax(z):\n    softact = np.exp(z - np.max(z, axis=1, keepdims=True))\n    softact=softact/np.sum(softact, axis=1, keepdims=True)\n    \n    return softact\n'

### Feed Forward Function :

In [15]:
def feedforward(X_train,W1,b1,W2,b2):
    z1= np.dot(X_train,W1)+b1
    h1=sigmoid(z1)
    
    z2=np.dot(h1,W2)+b2
    output=softmax(z2)
    return z1, h1, z2, output
        

### Error Function :

In [16]:
def error(ypred,y_train):     
    n_samples = y_train.shape[0]
    L = (ypred-y_train)/n_samples
    return L
    

### Cross Entropy Loss Function:

In [17]:

def crossentropy(ypred,y_train):
    L_sum = np.sum(np.multiply(y_train, np.log(ypred)))
    num_samples = y_train.shape[0]
    L = -(1./num_samples) * L_sum

    return L

### Function to calculate Sigmoid Derivative:

In [18]:
def sigmoid_derivative(h1):
    deriv=h1*(1-h1)
    return deriv

### Backpropagation Function to compute Gradient Information:

In [19]:
def backpropagation(h1,W2,ypred,y_train):
    
    #Calculate error
    L=error(ypred,y_train)
    
    # z1, h1, z2, output
    # z1, and z2 => dot products
    # h1 and output => activations
    change_output=L
    #dL/dW2
    delta_W2= np.dot(h1.T,change_output)
    
    #dL/db2
    delta_b2= np.sum(change_output,axis=0,keepdims=True)
    
    #dL/dh1
    delta_h1=np.dot(change_output,W2.T)

    change_h1= delta_h1*sigmoid_derivative(h1)
    
    #dL/dW1
    delta_W1= np.dot(X_train.T, change_h1)
    
    #dL/db1
    delta_b1= np.sum(change_h1,axis=0,keepdims=True)
    
    return delta_W2, delta_b2, delta_W1, delta_b1
    

### Function to update weights:

In [20]:
def update_weights(alpha,W1,b1,W2,b2,delta_W2,delta_b2,delta_W1,delta_b1):
    W2= W2- alpha*(delta_W2)
    b2= b2 - alpha*(delta_b2)
    W1= W1- alpha*(delta_W1)
    b1=b1- alpha*(delta_b1)
    
    return W2,b2,W1,b1

### Training Function:

In [29]:
def training():
    hNeurons=128                               #hidden neuron
    inputs=X_train.shape[1]                   #number of inputs
    oNeurons=y_train.shape[1]
    alpha= 0.05                        #learning rate
    epochs=300
    
    #retriving the actual labels for y_train:
    labely_train=np.argmax(y_train,axis=1)
    
    #initialize weights
    W1, b1, W2, b2= initialize_weights(inputs,hNeurons,oNeurons)
    
    for i in range(epochs):
        
        #feed forward
        dotprod1, h1, dotprod2, output=feedforward(X_train,W1,b1,W2,b2)

        #back propogate
        delta_W2, delta_b2, delta_W1, delta_b1= backpropagation(h1,W2,output,y_train)
        
        #update weights using gradient descent
        W2,b2,W1,b1=update_weights(alpha,W1,b1,W2,b2,delta_W2,delta_b2,delta_W1,delta_b1)

        #again feed forward using updated weights
        dotprod1, h1, dotprod2, output=feedforward(X_train,W1,b1,W2,b2)

        #Calculating Total Loss
        loss=crossentropy(output,y_train)                 # put in training func later
        print("Epoch {}: training loss = {}".format(i + 1,loss))
        
        
        
        #Retrieving the corresponding class labels:
        labelpred=np.argmax(output,axis=1)
        
        #Training Accuracy:
        accuracy = round((accuracy_score(labely_train, labelpred)*100),2)
        print("Training Accuracy after Epoch {} : {}%".format(i+1, accuracy))
        
    return W2,b2,W1,b1
        


In [30]:
W2,b2,W1,b1=training()

Epoch 1: training loss = 2.3513398081769155
Training Accuracy after Epoch 1 : 14.49%
Epoch 2: training loss = 2.189481571457147
Training Accuracy after Epoch 2 : 22.18%
Epoch 3: training loss = 2.069003881318173
Training Accuracy after Epoch 3 : 30.77%
Epoch 4: training loss = 1.9683907597418244
Training Accuracy after Epoch 4 : 37.63%
Epoch 5: training loss = 1.8813677534692541
Training Accuracy after Epoch 5 : 43.22%
Epoch 6: training loss = 1.8051914989300661
Training Accuracy after Epoch 6 : 47.92%
Epoch 7: training loss = 1.736564534157259
Training Accuracy after Epoch 7 : 52.12%
Epoch 8: training loss = 1.6734071611577965
Training Accuracy after Epoch 8 : 55.7%
Epoch 9: training loss = 1.6156318852939355
Training Accuracy after Epoch 9 : 59.05%
Epoch 10: training loss = 1.5632779428533272
Training Accuracy after Epoch 10 : 62.08%
Epoch 11: training loss = 1.5144180523953739
Training Accuracy after Epoch 11 : 64.78%
Epoch 12: training loss = 1.4693556533124985
Training Accuracy af

Epoch 96: training loss = 0.559565952525091
Training Accuracy after Epoch 96 : 88.28%
Epoch 97: training loss = 0.5566153424686655
Training Accuracy after Epoch 97 : 88.3%
Epoch 98: training loss = 0.553722304632462
Training Accuracy after Epoch 98 : 88.38%
Epoch 99: training loss = 0.5508912120832595
Training Accuracy after Epoch 99 : 88.37%
Epoch 100: training loss = 0.5480671451579768
Training Accuracy after Epoch 100 : 88.48%
Epoch 101: training loss = 0.5453312263588012
Training Accuracy after Epoch 101 : 88.47%
Epoch 102: training loss = 0.5425965423218181
Training Accuracy after Epoch 102 : 88.58%
Epoch 103: training loss = 0.5399410902580619
Training Accuracy after Epoch 103 : 88.54%
Epoch 104: training loss = 0.5372736120919948
Training Accuracy after Epoch 104 : 88.65%
Epoch 105: training loss = 0.5347442966441253
Training Accuracy after Epoch 105 : 88.64%
Epoch 106: training loss = 0.5321821483305256
Training Accuracy after Epoch 106 : 88.75%
Epoch 107: training loss = 0.529

Epoch 188: training loss = 0.40516723936708043
Training Accuracy after Epoch 188 : 90.65%
Epoch 189: training loss = 0.4041432292605096
Training Accuracy after Epoch 189 : 90.61%
Epoch 190: training loss = 0.40309953592698333
Training Accuracy after Epoch 190 : 90.68%
Epoch 191: training loss = 0.40202719847243445
Training Accuracy after Epoch 191 : 90.66%
Epoch 192: training loss = 0.40104681173791373
Training Accuracy after Epoch 192 : 90.7%
Epoch 193: training loss = 0.39998513087951776
Training Accuracy after Epoch 193 : 90.7%
Epoch 194: training loss = 0.3990645815253937
Training Accuracy after Epoch 194 : 90.72%
Epoch 195: training loss = 0.39802396476540225
Training Accuracy after Epoch 195 : 90.72%
Epoch 196: training loss = 0.39716791143297436
Training Accuracy after Epoch 196 : 90.77%
Epoch 197: training loss = 0.39617989260492475
Training Accuracy after Epoch 197 : 90.76%
Epoch 198: training loss = 0.3953509784914246
Training Accuracy after Epoch 198 : 90.79%
Epoch 199: trai

Epoch 280: training loss = 0.33973101471909434
Training Accuracy after Epoch 280 : 91.72%
Epoch 281: training loss = 0.33925142864087043
Training Accuracy after Epoch 281 : 91.69%
Epoch 282: training loss = 0.338598750282509
Training Accuracy after Epoch 282 : 91.74%
Epoch 283: training loss = 0.33803722968904737
Training Accuracy after Epoch 283 : 91.73%
Epoch 284: training loss = 0.33748716450794997
Training Accuracy after Epoch 284 : 91.77%
Epoch 285: training loss = 0.3370515510543793
Training Accuracy after Epoch 285 : 91.74%
Epoch 286: training loss = 0.3363509661779879
Training Accuracy after Epoch 286 : 91.78%
Epoch 287: training loss = 0.3359041267652244
Training Accuracy after Epoch 287 : 91.76%
Epoch 288: training loss = 0.33532395517997926
Training Accuracy after Epoch 288 : 91.79%
Epoch 289: training loss = 0.3348811101153694
Training Accuracy after Epoch 289 : 91.77%
Epoch 290: training loss = 0.33450582953796604
Training Accuracy after Epoch 290 : 91.81%
Epoch 291: train

### Evaluating our model on Testing data:

In [34]:
def testing(W2,b2,W1,b1,X_test,y_test):
    
    dotprod1, h1, dotprod2, output=feedforward(X_test,W1,b1,W2,b2)   
    labely_test=y_test
    #np.argmax(y_test,axis=1)
    labelpred=np.argmax(output,axis=1)
    accuracytest = round((accuracy_score(labely_test, labelpred)*100),2)        
    print("Accuracy on Testing data : {}%".format(accuracytest))
        

### Accuracy on Testing data :

In [35]:
result=testing(W2,b2,W1,b1,X_test,y_test)

Accuracy on Testing data : 90.87%
